In [167]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from helpers.least_squares import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [178]:
from helpers.proj1_helpers import *
# Aya : '/Users/mac/Documents/GitHub/ml-project-1-aaa_project1/data/train.csv' 
DATA_TRAIN_PATH = '/home/adeye/Documents/EPFL/MA1/ML_course/projects/ml-project-1-aaa_project1/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [169]:
from helpers.gradient_descent import *
from helpers.least_squares import *
from helpers.stochastic_gradient_descent import *
from helpers.ridge import *
from helpers.processing import *
from helpers.cross_validation import *
from helpers.feature_transformation import *

## Pre-processing our DataSet

In [170]:
jet_0,label_0,jet_1,label_1,jet_2_3, label_2_3= pre_process_data_pipeline(tX,y)

## Learning Algorithms After Cleaning Our DataSet

### Least Squares With Gradient Descent

jet_0,label_0,jet_1,label_1,jet_2_3, label_2_3= groupy_by_jet_num(tX,y)
    
remove_outliers(tX)
init_weights = np.ones((30,))
print(y.shape)
print(tX.shape)

weights,loss =  gradient_descent(y, tX,init_weights , 50000, .01)

print(weights)
print("{:e}".format(loss))

## Least Squares with OLS

weight,loss= least_squares(y,tX)
print(compute_squared_loss(y,tX,weight))

## Least Squares With Stochastic Gradient Descent 

remove_outliers(tX)
init_weights = np.ones((30,))
print(y.shape)
print(tX.shape)

weights,loss =  stochastic_gradient_descent(y, tX,init_weights , 500, 0.01, batch_size=1)

print(weights)
print(loss)

## Least Squares With Ridge Regression 

weights, loss = ridge(y, tX, 0.025)
print("{:e}".format(np.sqrt(2*loss)))

# Parameter Tuning

### Finding the best hyperparameters for ridge regression (degree and lambda)

In [171]:
# We use 10 fold cross-validation for this assignment as it gives us a good compromise (for our hardware) between low bias and good performance
k_fold = 10

### For each one of our subsets we find the best hyperparameters

In [172]:
#For jet_0 => find the best hyperparameters and find the weight that corresponds to those hyperparameters

[lowest_error_0,best_degree_0,best_lambda_0,rmse_tr,rmse_te] = cross_validation_ridge(k_fold,jet_0,label_0)

print("\n\n")
print("*******JET 0 *************")
print("Lowest_error:",lowest_error_0," Best degree:",best_degree_0," Best lambda:",best_lambda_0)
    

#X,Y = np.meshgrid(rmse_tr[:,0],rmse_tr[:,1])
#print(X)
#print(Y)

#print(rmse_tr[:,2])
#fig = plt.figure()
#ax = fig.add_subplot(projection='3d')
#ax.scatter(rmse_tr[:,0],rmse_tr[:,1],rmse_tr[:,2])
#plt.xlim([0, 1e+08])

#plt.show()
jet_0_extended = poly_expansion(jet_0,best_degree_0)
w_jet_0,_ = ridge(label_0,jet_0_extended,best_lambda_0)

print(w_jet_0.shape)

Degree: 0.0  Lambda: 1e-07  RMSE Training: 0.8718559628903682  RMSE Test: 0.8718523920593633
Degree: 1.0  Lambda: 1e-07  RMSE Training: 0.7377063950311074  RMSE Test: 0.7380711683005949
Degree: 2.0  Lambda: 1e-07  RMSE Training: 0.7130765956855225  RMSE Test: 1.3897213743532544
Degree: 3.0  Lambda: 1e-07  RMSE Training: 0.7007905657168225  RMSE Test: 1.2374116524923102
Degree: 4.0  Lambda: 1e-07  RMSE Training: 0.6959288240330819  RMSE Test: 2712.8479224220073
Degree: 5.0  Lambda: 1e-07  RMSE Training: 0.9652245235136585  RMSE Test: 405271.98746283853
Degree: 6.0  Lambda: 1e-07  RMSE Training: 5.939070955442334  RMSE Test: 13025329.090063512
Degree: 7.0  Lambda: 1e-07  RMSE Training: 23.986333598086905  RMSE Test: 215004886.11787271
Degree: 8.0  Lambda: 1e-07  RMSE Training: 2077404.7474475757  RMSE Test: 46359339282.487915
Degree: 9.0  Lambda: 1e-07  RMSE Training: 13170744.523042297  RMSE Test: 16010273068497.037
Degree: 0.0  Lambda: 1e-06  RMSE Training: 0.8718559628909128  RMSE Tes

In [173]:
#For jet_1 => find the best hyperparameters and find the weight that corresponds to those hyperparameters

[lowest_error_1,best_degree_1,best_lambda_1,rmse_tr,rmse_te] = cross_validation_ridge(k_fold,jet_1,label_1)
print("\n\n")
print("*******JET 1 *************")
print("Lowest_error:",lowest_error_1," Best degree:",best_degree_1," Best lambda:",best_lambda_1)

jet_1_extended = poly_expansion(jet_1,best_degree_1)
w_jet_1,_ = ridge(label_1,jet_1_extended,best_lambda_1)

print(w_jet_1.shape)

Degree: 0.0  Lambda: 1e-07  RMSE Training: 0.9584450111432654  RMSE Test: 0.958463428207593
Degree: 1.0  Lambda: 1e-07  RMSE Training: 0.8632964889818588  RMSE Test: 0.8635846708506504
Degree: 2.0  Lambda: 1e-07  RMSE Training: 0.8334061642407902  RMSE Test: 0.8345179074773321
Degree: 3.0  Lambda: 1e-07  RMSE Training: 0.81624333928636  RMSE Test: 0.8219852518359223
Degree: 4.0  Lambda: 1e-07  RMSE Training: 0.806101215946956  RMSE Test: 0.8222089435885938
Degree: 5.0  Lambda: 1e-07  RMSE Training: 0.8008871275776878  RMSE Test: 0.8071947829219605
Degree: 6.0  Lambda: 1e-07  RMSE Training: 0.7946266156834191  RMSE Test: 0.8437188420491293
Degree: 7.0  Lambda: 1e-07  RMSE Training: 0.7848271875101357  RMSE Test: 1.2580566848524606
Degree: 8.0  Lambda: 1e-07  RMSE Training: 0.7749472172926022  RMSE Test: 2.3264335979806874
Degree: 9.0  Lambda: 1e-07  RMSE Training: 0.7706629314416529  RMSE Test: 4.408002340027459
Degree: 0.0  Lambda: 1e-06  RMSE Training: 0.9584450111434336  RMSE Test: 0

In [174]:
#For jet_2_3 => find the best hyperparameters and find the weight that corresponds to those hyperparameters

[lowest_error_2_3,best_degree_2_3,best_lambda_2_3,rmse_tr,rmse_te] = cross_validation_ridge(k_fold,jet_2_3,label_2_3)

print("\n\n")
print("*******JET 2_3 *************")
print("Lowest_error:",lowest_error_2_3," Best degree:",best_degree_2_3," Best lambda:",best_lambda_2_3)

jet_2_3_extended = poly_expansion(jet_2_3,best_degree_2_3)
w_jet_2_3,_ = ridge(label_2_3,jet_2_3_extended,best_lambda_2_3)

print(w_jet_2_3.shape)

Degree: 0.0  Lambda: 1e-07  RMSE Training: 0.9944759461700402  RMSE Test: 0.9944798175507181
Degree: 1.0  Lambda: 1e-07  RMSE Training: 0.8498488074937182  RMSE Test: 0.8502854000440656
Degree: 2.0  Lambda: 1e-07  RMSE Training: 0.8316080381404045  RMSE Test: 0.8324954708330303
Degree: 3.0  Lambda: 1e-07  RMSE Training: 0.8068073066289368  RMSE Test: 0.8090088856985981
Degree: 4.0  Lambda: 1e-07  RMSE Training: 0.7890532717947665  RMSE Test: 0.7970387612984328
Degree: 5.0  Lambda: 1e-07  RMSE Training: 0.7841368653010241  RMSE Test: 0.801666559037433
Degree: 6.0  Lambda: 1e-07  RMSE Training: 0.7774009662002904  RMSE Test: 1.0014903471087893
Degree: 7.0  Lambda: 1e-07  RMSE Training: 0.7615315050984078  RMSE Test: 0.9344830921399175
Degree: 8.0  Lambda: 1e-07  RMSE Training: 0.7452866383354437  RMSE Test: 1.6413923221003077
Degree: 9.0  Lambda: 1e-07  RMSE Training: 0.738624463295954  RMSE Test: 6.96370101675611
Degree: 0.0  Lambda: 1e-06  RMSE Training: 0.9944759461700621  RMSE Test: 

In [175]:
#cross_validation_log_visualization(degrees,rmse_tr,rmse_te)
#print(rmse_tr)
#print(rmse_te)

degree = 5
lambda_ = 1e-07
poly = poly_expansion(jet_0,degree)
weight,loss = ridge(label_0, poly, lambda_)

print(loss)

0.24330405912727762


## Generate predictions and save ouput in csv format for submission:

In [185]:
DATA_TEST_PATH = '/home/adeye/Documents/EPFL/MA1/ML_course/projects/ml-project-1-aaa_project1/test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
print(tX_test.shape)


(568238, 30)


In [203]:
#Because we have split our data into three subsets we get 3 different weight vectors with different dimensions. 
#We therefore have to create the same split for our test set to be able to multiply it by the weight vectors
jet_0_test,indices_0,jet_1_test,indices_1,jet_2_3_test,indices_2_3 = split_test_set(tX_test,best_degree_0,best_degree_1,best_degree_2_3)

y_pred_jet_0 = predict_labels(w_jet_0,jet_0_test).reshape((len(y_pred_jet_0),1))
y_pred_jet_1 = predict_labels(w_jet_1,jet_1_test).reshape((len(y_pred_jet_1),1))
y_pred_jet_2_3 = predict_labels(w_jet_2_3,jet_2_3_test).reshape((len(y_pred_jet_2_3),1))

y_pred = np.zeros((tX_test.shape[0],1))

indices_0 = indices_0.reshape(-1,)
indices_1 = indices_1.reshape(-1,)
indices_2_3 = indices_2_3.reshape(-1,)

y_pred[indices_0] = y_pred_jet_0
y_pred[indices_1] = y_pred_jet_1
y_pred[indices_2_3] = y_pred_jet_2_3

print(y_pred)


[[-1.]
 [-1.]
 [-1.]
 ...
 [ 1.]
 [-1.]
 [-1.]]


In [204]:
OUTPUT_PATH = '/home/adeye/Documents/EPFL/MA1/ML_course/projects/ml-project-1-aaa_project1/output.csv' 
#y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)